# Chat Writer

This notebook provides a quick overview for getting started with Writer [chat](/docs/concepts/chat_models/).

Writer has several chat models. You can find information about their latest models and their costs, context windows, and supported input types in the [Writer docs](https://dev.writer.com/home).


## Overview

### Integration details
| Class                                                                                                                                         | Package          | Local | Serializable | JS support |                                        Package downloads                                         |                                        Package latest                                         |
|:----------------------------------------------------------------------------------------------------------------------------------------------|:-----------------| :---: | :---: |:----------:|:------------------------------------------------------------------------------------------------:|:---------------------------------------------------------------------------------------------:|
| [ChatWriter](https://python.langchain.com/v0.2/api_reference/community/tools/langchain_community.tools.langchain_writer.tool.ChatWriter.html) | [langchain-writer](https://api.python.langchain.com/en/latest/community_api_reference.html) |      ❌       |                                       ❌                                       | ❌ | ![PyPI - Downloads](https://img.shields.io/pypi/dm/langchain-writer?style=flat-square&label=%20) | ![PyPI - Version](https://img.shields.io/pypi/v/langchain-writer?style=flat-square&label=%20) |
### Model features
| [Tool calling](/docs/how_to/tool_calling) | Structured output | JSON mode | Image input | Audio input | Video input | [Token-level streaming](/docs/how_to/chat_streaming/) | Native async |         [Token usage](/docs/how_to/chat_token_usage_tracking/)          | Logprobs |
| :---: |:-----------------:| :---: | :---: |  :---: | :---: | :---: | :---: |:--------------------------------:|:--------:|
| ✅ |         ❌         | ❌ | ❌ | ❌ | ❌ | ✅ | ✅ |                ✅                 |    ❌     |

### Credentials

Head to [Writer AI Studio](https://app.writer.com/aistudio/signup?utm_campaign=devrel) to sign up and generate an API key. Once you've done this set the WRITER_API_KEY environment variable. It's necessary to create ChatWriter instance:

In [ ]:
import getpass
import os

if not os.getenv("WRITER_API_KEY"):
    os.environ["WRITER_API_KEY"] = getpass.getpass("Enter your Writer API key: ")

If you want to get automated tracing of your model calls you can also set your [LangSmith](https://docs.smith.langchain.com/) API key by uncommenting below:

In [ ]:
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")

### Installation

The LangChain Writer integration lives in the `langchain-writer` package:

In [ ]:
%pip install -qU langchain-writer

## Instantiation

Now we can instantiate our model object and generate chat completions:

In [ ]:
from langchain_writer import ChatWriter

llm = ChatWriter(
    model="palmyra-x-004",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

## Invocation

In [ ]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

In [ ]:
print(ai_msg.content)

## Streaming

In [ ]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming. Sing a song about it"),
]
ai_stream = llm.stream(messages)
ai_stream

In [ ]:
for chunk in ai_stream:
    print(chunk.content, end="")

## Tool calling

Writer supports [tool calling](https://dev.writer.com/api-guides/tool-calling), which lets you describe tools and their arguments, and have the model return a JSON object with a tool to invoke and the inputs to that tool.

### ChatWriter.bind_tools()

With `ChatWriter.bind_tools`, we can easily pass in Pydantic classes, dict schemas, LangChain tools, or even functions as tools to the model. Under the hood these are converted to tool schemas, which looks like:
```
{
    "name": "...",
    "description": "...",
    "parameters": {...}  # JSONSchema
}
```
and passed in every model invocation.

In [ ]:
from pydantic import BaseModel, Field


class GetWeather(BaseModel):
    """Get the current weather in a given location"""

    location: str = Field(..., description="The city and state, e.g. San Francisco, CA")


llm.bind_tools([GetWeather])

In [ ]:
ai_msg = llm.invoke(
    "what is the weather like in New York City",
)
ai_msg

In [ ]:
print(ai_msg.tool_calls)

## Important feature

ChatWriter ___bind_tools()___ method have specific functionality: it doesn't create new instance with binded tools, but store received tools and tool_choice in initial class instance attributes, to pass them as parameters during Palmyra LLM call while Chat invocation. Pay attention about it while using ChatWriter class with tools calling. This approach allows to support different tool types e.g. __function__ and __graph__. __Graph__ - is one of remotely called Writer Palmyra tools type. For further information visit our [docs](https://dev.writer.com/api-guides/knowledge-graph#knowledge-graph). For more information about tools usage visit tools docs.

## Batching

In [ ]:
ai_batch = llm.batch(
    [
        "How to cook pancakes?",
        "How to compose poem?",
        "How to run faster?",
    ],
    config={"max_concurrency": 3},
)
ai_batch

In [ ]:
for batch in ai_batch:
    print(batch.content)
    print("-"*100)

## Async

All features above (invocation, streaming, batching, tools calling) also support asynchronous usage!

## Chaining

You can [chain](/docs/how_to/sequence/) our model with a prompt template like so:


In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
chain.invoke(
    {
        "input_language": "English",
        "output_language": "German",
        "input": "I love programming.",
    }
)

## API reference

For detailed documentation of all ChatWriter features and configurations head to the [API reference](https://python.langchain.com/api_reference/writer/chat_models/langchain_writer.chat_models.ChatWriter.html#langchain_writer.chat_models.ChatWriter)